# Analise de finanças no mês
### O objetivo deste notebook é criar uma lógica de como analisar o saldo de um mês através de extratos e resumos de fatura,

O meu saldo mensal é de R$ 2896, o que eu gasto em média em contas é R$574, minha menta de aporte mensal é de 1500

In [1]:
# Esta expressão representa o que geralmente eu tenho líquido no mês para gastos comigo mesmo.
(2986-574-1500)

912

In [2]:
#import polars as pl
import pandas as pd
import os
import sys

In [5]:
''' 
 Para extrair o saldo do mês eu preciso analisar o que eu tenho de dívidas no crédito no mês, e considerar todas as transações feitas no débito
 representamos esses dados em dois dataframes, um com as faturas e outro com as transações. 
 Consideramos também o período contábil para análise, que é de um mês, caso a analise seja realizada no começo do mês, considera-se os valores
 a receber.
'''
# TODO Depois irei parametrizar os arquivos para serem passados como argumentos no momento de executar o programa.
# Busca pelos extratos nos diretorios esperados (./extratos/mêsAno) exemplo:(janeiro2024)
df_extrato = pd.read_csv("../extratos/maio24/NU_835802811_01MAI2024_25MAI2024.csv",sep=",",encoding='ISO-8859-1')
df_resumo_fatura = pd.read_csv("../extratos/junho2024/nubank-2024-06.csv",sep=",",encoding='ISO-8859-1')

In [12]:
df_extrato.head()

,Data,Valor,DescriÃ§Ã£o
0,01/05/2024,-10.58,Compra no dÃ©bito - Supermercados Boa
1,02/05/2024,-3.99,Compra no dÃ©bito - Irmaos Boa
2,02/05/2024,-12.97,Compra no dÃ©bito - So Ofertas Jundiai
3,02/05/2024,-261.21,TransferÃªncia enviada pelo Pix - Thiago Nagat...
4,03/05/2024,-11.12,Compra no dÃ©bito - Irmaos Boa


In [23]:
df_extrato['DescriÃ§Ã£o'] = df_extrato['DescriÃ§Ã£o'].str.replace('Compra de AÃ§Ãµes', 'compra acoes').str.replace('Compra no dÃ©bito', 'compra debito')

,Data,Valor,DescriÃ§Ã£o
0,01/05/2024,-10.58,compra debito - Supermercados Boa
1,02/05/2024,-3.99,compra debito - Irmaos Boa
2,02/05/2024,-12.97,compra debito - So Ofertas Jundiai
3,02/05/2024,-261.21,TransferÃªncia enviada pelo Pix - Thiago Nagat...
4,03/05/2024,-11.12,compra debito - Irmaos Boa
5,03/05/2024,-24.95,compra debito - So Ofertas Jundiai
6,04/05/2024,-21.00,compra debito - Pag*Contatojgcenter
7,04/05/2024,-11.38,compra debito - Supermercados Boa
8,04/05/2024,-54.00,TransferÃªncia enviada pelo Pix - Thiago Nagat...
9,04/05/2024,-13.94,compra debito - Supermercados Boa


In [32]:
for a,b in df_extrato.iterrows():
    print(b)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [20]:
df = pd.DataFrame({'Teste':[12,23,23],'Teste2':[19,20,21],'Teste3':[1,2,3]})
index_ = ['Row1','Row2','Row3']

,Teste,Teste2,Teste3
0,12,19,1
1,23,20,2
2,23,21,3


In [5]:
# Contando transações listadas em faturas por categoria.
transportes = 0
supermercado = 0
for i in df_resumo_fatura['category']:
    if(i=='transporte'):
        transportes += 1
    if(i=='supermercado'):
        supermercado += 1

print(transportes)
print(supermercado)


31
18


In [6]:
print(df_resumo_fatura['category']);

0       transporte
1        serviÃ§os
2       transporte
3        serviÃ§os
4       transporte
          ...     
71    supermercado
72      transporte
73         payment
74      transporte
75      transporte
Name: category, Length: 76, dtype: object
